# <center>Artificial Neural Networks</center>
---

### <center>Import required libraries</center>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
tf.__version__

'2.9.1'

## <center>Part I: Data Preprocessing</center>

### <center>Importing dataset</center>

In [3]:
dataset = pd.read_csv("datasets/Churn_Modelling.csv")
X = dataset.iloc[ : , 3 : -1].values
y = dataset.iloc[ : , -1].values

In [4]:
print(X, end = "\n\n")
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]

[1 0 1 ... 1 1 0]


### <center>Encoding categorical data</center>

In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[ : , 2] = le.fit_transform(X[ : , 2])

In [6]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### <center>One hot encoding the "Geography" column</center>

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers = [("encoder", OneHotEncoder(), [1])],
                       remainder = "passthrough")
X = np.array(ct.fit_transform(X))

In [8]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### <center>Splitting the dataset into training and testing sets</center>

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state = 0)

### <center>Feature Scaling</center>

In [10]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## <center>Part II : Building the Neural Network</center>

### <center>Initializing the neural network</center>

In [11]:
neural_network = keras.models.Sequential()

### <center>Add First hidden layer</center>

In [12]:
neural_network.add(keras.layers.Dense(units = 6, activation = "relu"))

### <center>Add second hidden layer</center>

In [13]:
neural_network.add(keras.layers.Dense(units = 6, activation = "relu"))

### <center>Add output layer</center>

In [14]:
neural_network.add(keras.layers.Dense(units = 1, activation = "sigmoid"))

### <center>Compiling the neural network</center>

In [15]:
neural_network.compile(optimizer = "adam",
                       loss = "binary_crossentropy",
                       metrics = ['accuracy'])

## <center>Part III: Training the neural network</center>

### <center>Training the neural network</center>

In [16]:
neural_network.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 5s 9ms/step - loss: 0.6141 - accuracy: 0.7032
Epoch 2/100
250/250 [==============================] - 3s 10ms/step - loss: 0.4843 - accuracy: 0.7968
Epoch 3/100
250/250 [==============================] - 2s 9ms/step - loss: 0.4436 - accuracy: 0.8046
Epoch 4/100
250/250 [==============================] - 2s 9ms/step - loss: 0.4265 - accuracy: 0.8106
Epoch 5/100
250/250 [==============================] - 3s 11ms/step - loss: 0.4170 - accuracy: 0.8181
Epoch 6/100
250/250 [==============================] - 3s 10ms/step - loss: 0.4081 - accuracy: 0.8220
Epoch 7/100
250/250 [==============================] - 2s 10ms/step - loss: 0.3996 - accuracy: 0.8291
Epoch 8/100
250/250 [==============================] - 2s 10ms/step - loss: 0.3917 - accuracy: 0.8361
Epoch 9/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3838 - accuracy: 0.8419
Epoch 10/100
250/250 [==============================] - 3s 10ms/step - loss: 0.3764 - 

## <center>Part IV - Making the predictions and evaluating the model</center>

Use the model to predict if the customer with the following informations will leave the bank: 

+ **Geography**: France
+ **Credit Score**: 600
+ **Gender**: Male
+ **Age**: 40 years old
+ **Tenure**: 3 years
+ **Balance**: US \$60000
+ **Number of Products**: 2
+ **Does this customer have a credit card?** Yes
+ **Is this customer an Active Member**: Yes
+ **Estimated Salary**: US \$50000

So, should we say goodbye to that customer?

In [17]:
outcome = neural_network.predict(
    sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

if outcome < 0.5:
    print("Customer stays")
else:
    print("Customer leaves")

1/1 [==============================] - 1s 678ms/step
Customer stays


Therefore, the model predicts that this customer stays in the bank!

**Notes:**
1. Notice that the values of the features were all input in a double pair of square brackets. That's because the `predict` method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

2. Notice also that the `France` country was not inputted as a string in the last column but as `1, 0, 0`. That's because `predict` method expects the one-hot-encoded values of the state. We see that `France` is encoded as `1, 0, 0` in the first row of the matrix of features X. Take care to include these values in the first three columns, as the dummy variables are always created in the first columns.

### <center>Predicting the test dataset results</center>

In [18]:
y_pred = neural_network.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),
                      y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 1s 8ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### <center>Making the confusion matrix</center>

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1538   57]
 [ 212  193]]


0.8655

### <center>Saving the neural network for future use</center>

In [20]:
neural_network.save("neural_network.h5")